In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from imblearn.under_sampling import TomekLinks

In [53]:
data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_6\data\german.data-numeric', header = None, delim_whitespace = True)

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Admin\AppData\Local\Temp\ipykernel_19664\241190258.py:1: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_6\data\german.data-numeric', header = None, delim_whitespace = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_19664\241190258.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv('C:\Th-th-ch-BAI-Labs\Bai_6\data\german.data-numeric', header = None, delim_whitespace = True)


In [54]:
data.drop(columns = [18, 14], inplace = True)

In [55]:
data[23].value_counts()

23
1    630
0    370
Name: count, dtype: int64

In [56]:
target = data.pop(24)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [58]:
smote = TomekLinks()
X_res, y_res = smote.fit_resample(X_train, y_train)

In [59]:
varthre = VarianceThreshold(threshold = 0.02)
varthre.fit(X_train)

,threshold,0.02


In [60]:
c = X_train.columns[~varthre.get_support()]

In [61]:
print(c)

Index([], dtype='int64')


In [62]:
varthre.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [63]:
def find_best_hyperparameter(model: BaseEstimator,para: dict, X: pd.DataFrame, y: pd.Series) -> None:
    grid = GridSearchCV(model, param_grid = para, cv = 5, scoring = 'f1')
    grid.fit(X, y)
    print(f'Best estimator: {grid.best_estimator_}')
    print(f'Best score: {grid.best_score_}')

In [64]:
param_grid_lgr = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 500, 1000],
    'class_weight': [None, 'balanced']
}
param_grid_gnb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}
param_grid_rf = {
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(1, 10, 2)) + [None],
    'min_samples_leaf': range(1, 10, 2),
    'min_samples_split': range(1, 10, 2),
}
param_grid_knn = {
    'n_neighbors': range(1, 31),
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'manhattan'],
    'p': [1, 2]
}

In [65]:
#find_best_hyperparameter(KNeighborsClassifier(), param_grid_knn, X_train, y_train)

In [66]:
scaler = StandardScaler()

In [67]:
def score_model(model: BaseEstimator, X: pd.DataFrame, y: pd.Series) -> None:
    model.fit(X, y)
    print(f'=============={model.__class__.__name__}==============')
    print(f'Recall score: {recall_score(model.predict(X_test), y_test):.02}')
    print(f'Precision score: {precision_score(model.predict(X_test), y_test):.02}')
    print(f'Accuracy score: {accuracy_score(model.predict(X_test), y_test):.02}')
    print(f'F1 score: {f1_score(model.predict(X_test), y_test):.02}')


In [68]:
score_model(LogisticRegression(C=10, max_iter=1000, penalty='l1', solver='saga'), X_train, y_train)
score_model(GaussianNB(var_smoothing=1e-05),X_train, y_train)
score_model(RandomForestClassifier(n_estimators = 150, criterion='entropy', max_depth=10, min_samples_split=4, random_state = 42), X_train, y_train)
score_model(SVC(kernel = 'rbf', C = 1, gamma = 'scale', class_weight = 'balanced'), scaler.fit_transform(X_train), y_train)
score_model(KNeighborsClassifier(n_neighbors=14, p=1, weights='distance'), scaler.fit_transform(X_train), y_train)

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


==============LogisticRegression==============
Recall score: 0.79
Precision score: 0.91
Accuracy score: 0.77
F1 score: 0.84
==============GaussianNB==============
Recall score: 0.81
Precision score: 0.82
Accuracy score: 0.74
F1 score: 0.82
==============RandomForestClassifier==============
Recall score: 0.79
Precision score: 0.95
Accuracy score: 0.79
F1 score: 0.86
==============SVC==============
Recall score: 0.7
Precision score: 1.0
Accuracy score: 0.7
F1 score: 0.83
==============KNeighborsClassifier==============
Recall score: 0.7
Precision score: 1.0
Accuracy score: 0.7
F1 score: 0.83


In [69]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,15,16,17,19,20,21,22,23
0,1,6,4,12,5,5,3,4,1,67,...,1,2,0,0,1,0,1,0,0,1
1,2,48,2,60,1,3,2,2,1,22,...,1,1,0,0,1,0,1,0,0,1
2,4,12,4,21,1,4,3,3,1,49,...,2,1,0,0,1,0,1,0,1,0
3,1,42,2,79,1,4,3,4,2,45,...,2,1,0,0,0,0,0,0,0,1
4,1,24,3,49,1,3,3,4,4,53,...,2,1,1,0,1,0,0,0,0,1
